# Assignment 2

This notebook is intended to produce the plots and figures for the report on Problem 1 of the practical. You should not run this notebook in Google Colab until you have finished constructing the correct solutions for transformer_solution.py and encoder_decoder_solution.py

This notebook provides some limited commentary on several HuggingFace Features and toolage. You will use HuggingFace Datasets to load the Amazon Polarity dataset for sentiment analysis. The notebook will define a Bert tokenizer, collate functions, and then train and evaluate several models using the HuggingFace utilities mentioned above. Remember, the most crucial part here is running the experiments for the report.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
# allows plots to be displayed directly in the notebook rather than in a separate window
#  loads the autoreload extension : enables automatic reloading of modules before executing code
# if there are changes in a module that has been imported, those changes will be automatically reflected without needing to restart the kernel
# set up of automatic reloading, more aggressive approach than %autoreload 1
# autoreload 2 reload all modules except those excluded by being explicitly marked to be ignored or excluded

### Mount your Google Drive

### Link your assignment folder & install requirements
Enter the path to the assignment folder in your Google Drive
If you run this notebook locally or on a cluster (i.e. not on Google Colab)
you can delete this cell which is specific to Google Colab.

In [2]:
import sys
import os
import shutil
import warnings
import json

folder = "" #@param {type:"string"}
#!ln -Ts "$folder" /content/deepL_assignment2> /dev/null
#!cp gdrive/MyDrive/deepL_assignment2/transformer.py .
#!cp gdrive/MyDrive/deepL_assignment2/lstm.py .

# Add the assignment folder to Python path
if '/content/assignment' not in sys.path:
  sys.path.insert(0, '/content/assignment')

# Check if CUDA is available
import torch
if not torch.cuda.is_available():
  warnings.warn('CUDA is not available.')

/var/folders/s0/9my1yr7920lfy9vmb78bvjch0000gn/T/ipykernel_10858/2122472430.py:19: UserWarning: CUDA is not available.
  warnings.warn('CUDA is not available.')


### Running on GPU
For this assignment, it will be necessary to run your experiments on GPU. To make sure the notebook is running on GPU, you can change the notebook settings with
* (EN) `Edit > Notebook Settings`
* (FR) `Modifier > Paramètres du notebook`


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from sklearn.metrics import f1_score, accuracy_score
import time

from typing import List, Dict, Union, Optional, Tuple
import torch

from dataclasses import dataclass
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm.auto import tqdm


from datasets import Dataset
import transformers

from datasets import load_dataset
from tokenizers import Tokenizer

from transformer import Transformer, MultiHeadedAttention
from lstm import EncoderDecoder

/Users/agathefernandesmachado/Downloads/deepL_assignment2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset_train = load_dataset("yelp_polarity", split="train", cache_dir="assignment/data")
dataset_test = load_dataset("yelp_polarity", split="test[:1000]", cache_dir="assignment/data")

### 🔍 Quick look at the data
Lets have quick look at a few samples in our test set.

In [5]:
n_samples_to_see = 3
for i in range(n_samples_to_see):
  print("-"*30)
  print("title:", dataset_test[i]["text"])
  print("label:", dataset_test[i]["label"])

------------------------------
title: Contrary to other reviews, I have zero complaints about the service or the prices. I have been getting tire service here for the past 5 years now, and compared to my experience with places like Pep Boys, these guys are experienced and know what they're doing. \nAlso, this is one place that I do not feel like I am being taken advantage of, just because of my gender. Other auto mechanics have been notorious for capitalizing on my ignorance of cars, and have sucked my bank account dry. But here, my service and road coverage has all been well explained - and let up to me to decide. \nAnd they just renovated the waiting room. It looks a lot better than it did in previous years.
label: 1
------------------------------
title: Last summer I had an appointment to get new tires and had to wait a super long time. I also went in this week for them to fix a minor problem with a tire they put on. They \""fixed\"" it for free, and the very next morning I had the 

### 1️. Tokenize the `text`
Tokenize the `text`portion of each sample (i.e. parsing the text to smaller chunks). Tokenization can happen in many ways; traditionally, this was done based on the white spaces. With transformer-based models, tokenization is performed based on the frequency of occurrence of "chunk of text". This frequency can be learned in many different ways. However the most common one is the [**wordpiece**](https://arxiv.org/pdf/1609.08144v2.pdf) model.
> The wordpiece model is generated using a data-driven approach to maximize the language-model likelihood
of the training data, given an evolving word definition. Given a training corpus and a number of desired
tokens $D$, the optimization problem is to select $D$ wordpieces such that the resulting corpus is minimal in the
number of wordpieces when segmented according to the chosen wordpiece model.

Under this model:
1. Not all things can be converted to tokens depending on the model. For example, most models have been pretrained without any knowledge of emojis. So their token will be `[UNK]`, which stands for unknown.
2. Some words will be mapped to multiple tokens!
3. Depending on the kind of model, your tokens may or may not respect capitalization

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [7]:
input_sample = "Welcome to IFT6135. We now teach you 🤗(HUGGING FACE) Library :DDD."
tokenizer.tokenize(input_sample)

['welcome',
 'to',
 'if',
 '##t',
 '##6',
 '##13',
 '##5',
 '.',
 'we',
 'now',
 'teach',
 'you',
 '[UNK]',
 '(',
 'hugging',
 'face',
 ')',
 'library',
 ':',
 'dd',
 '##d',
 '.']

### 2. Encoding
Once we have tokenized the text, we then need to convert these chuncks to numbers so we can feed them to our model. This conversion is basically a look-up in a dictionary **from `str` $\to$ `int`**. The tokenizer object can also perform this work. While it does so it will also add the *special* tokens needed by the model to the encodings.

In [8]:
input_sample = "Welcome to IFT6135. We now teach you 🤗(HUGGING FACE) Library :DDD." #@param {type: "string"}

print("--> Token Encodings:\n",tokenizer.encode(input_sample))
print("-."*15)
print("--> Token Encodings Decoded:\n",tokenizer.decode(tokenizer.encode(input_sample)))

--> Token Encodings:
 [101, 6160, 2000, 2065, 2102, 2575, 17134, 2629, 1012, 2057, 2085, 6570, 2017, 100, 1006, 17662, 2227, 1007, 3075, 1024, 20315, 2094, 1012, 102]
-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.
--> Token Encodings Decoded:
 [CLS] welcome to ift6135. we now teach you [UNK] ( hugging face ) library : ddd. [SEP]


### 3️. Truncate/Pad samples
Since all the sample in the batch will not have the same sequence length, we would need to truncate the longer sequences (i.e. the ones that exeed a predefined maximum length) and pad the shorter ones so we that we can equal length for all the samples in the batch. Once this is achieved, we would need to convert the result to `torch.Tensor`s and return. These tensors will then be retrieved from the [dataloader](https://https//pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader).

In [9]:
class Collate:
    def __init__(self, tokenizer: str, max_len: int) -> None:
        self.tokenizer_name = tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(self.tokenizer_name)
        self.max_len = max_len

    def __call__(self, batch: List[Dict[str, Union[str, int]]]) -> Dict[str, torch.Tensor]:
        texts = list(map(lambda batch_instance: batch_instance["text"], batch))
        tokenized_inputs = self.tokenizer(
            texts,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt",
            return_token_type_ids=False,
        )

        labels = list(map(lambda batch_instance: int(batch_instance["label"]), batch))
        labels = torch.LongTensor(labels)
        return dict(tokenized_inputs, **{"labels": labels})

#### 🧑‍🍳 Setting up the collate function

In [10]:
tokenizer_name = "bert-base-uncased"
sample_max_length = 256
collate = Collate(tokenizer=tokenizer_name, max_len=sample_max_length)

### 4. Models

In [11]:
from transformers import AutoModel
import torch.nn as nn
torch.random.manual_seed(0)

class ReviewClassifier(nn.Module):
    def __init__(self, backbone: str, backbone_hidden_size: int, nb_classes: int):
        super(ReviewClassifier, self).__init__()
        self.backbone = backbone
        self.backbone_hidden_size = backbone_hidden_size
        self.nb_classes = nb_classes
        self.back_bone = AutoModel.from_pretrained(
            self.backbone,
            output_attentions=False,
            output_hidden_states=False,
        )
        self.classifier = torch.nn.Linear(self.backbone_hidden_size, self.nb_classes)

    def forward(
        self, input_ids: torch.Tensor, attention_mask: torch.Tensor, labels: Optional[torch.Tensor] = None
    ) -> Union[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        back_bone_output = self.back_bone(input_ids, attention_mask=attention_mask)
        hidden_states = back_bone_output[0]
        pooled_output = hidden_states[:, 0]  # getting the [CLS] token
        logits = self.classifier(pooled_output)
        if labels is not None:
            loss_fn = torch.nn.CrossEntropyLoss()
            loss = loss_fn(
                logits.view(-1, self.nb_classes),
                labels.view(-1),
            )
            return loss, logits
        return logits

class ReviewClassifierLSTM(nn.Module):
    def __init__(self, nb_classes: int, encoder_only: bool = False,
        with_attn: bool = True, dropout: int = 0.5, hidden_size: int = 256):
        super(ReviewClassifierLSTM, self).__init__()
        self.nb_classes = nb_classes
        self.encoder_only = encoder_only

        if with_attn:
            attn = MultiHeadedAttention(head_size = 2*hidden_size, num_heads=1)
        else:
            attn = None

        self.back_bone = EncoderDecoder(dropout=dropout, encoder_only=encoder_only,
                                        attn=attn, hidden_size=hidden_size)

        if self.encoder_only:
            self.classifier = torch.nn.Linear(hidden_size*2, self.nb_classes)
        else:
            self.classifier = torch.nn.Linear(hidden_size, self.nb_classes)

    def forward(
        self, input_ids: torch.Tensor, attention_mask: torch.Tensor, labels: Optional[torch.Tensor] = None
    ) -> Union[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        pooled_output, _ = self.back_bone(input_ids, attention_mask)
        logits = self.classifier(pooled_output)
        if labels is not None:
            loss_fn = torch.nn.CrossEntropyLoss()
            loss = loss_fn(
                logits.view(-1, self.nb_classes),
                labels.view(-1),
            )
            return loss, logits
        return logits


class ReviewClassifierTransformer(nn.Module):
    def __init__(self, nb_classes: int, num_heads: int = 4, num_layers: int = 4, block: str="prenorm", dropout: float = 0.3):
        super(ReviewClassifierTransformer, self).__init__()
        self.nb_classes = nb_classes
        self.back_bone = Transformer(num_heads=num_heads, num_layers=num_layers, block=block, dropout=dropout)
        self.classifier = torch.nn.Linear(256, self.nb_classes)

    def forward(
        self, input_ids: torch.Tensor, attention_mask: torch.Tensor, labels: Optional[torch.Tensor] = None
    ) -> Union[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        attention_mask = torch.cat([torch.ones(attention_mask.shape[0]).unsqueeze(1).to(device),
                                    attention_mask], dim=1)
        back_bone_output = self.back_bone(input_ids, attention_mask)
        hidden_states = back_bone_output
        pooled_output = hidden_states
        logits = self.classifier(pooled_output)
        if labels is not None:
            loss_fn = torch.nn.CrossEntropyLoss()
            loss = loss_fn(
                logits.view(-1, self.nb_classes),
                labels.view(-1),
            )
            return loss, logits
        return logits

### 5. Trainer

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"--> Device selected: {device}")
def train_one_epoch(
    model: torch.nn.Module, training_data_loader: DataLoader, optimizer: torch.optim.Optimizer, logging_frequency: int, testing_data_loader: DataLoader, logger: dict):
    model.train()
    optimizer.zero_grad()
    epoch_loss = 0
    logging_loss = 0
    start_time = time.time()
    mini_start_time = time.time()
    for step, batch in enumerate(training_data_loader):
        batch = {key: value.to(device) for key, value in batch.items()}
        outputs = model(**batch)
        loss = outputs[0]
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        logging_loss += loss.item()

        if (step + 1) % logging_frequency == 0:
            freq_time = time.time()-mini_start_time
            logger['train_time'].append(freq_time+logger['train_time'][-1])
            logger['train_losses'].append(logging_loss/logging_frequency)
            logger['train_torch.cuda.memory_allocated'] = torch.cuda.memory_allocated(0)/1024/1024/1024 # GB
            logger['train_torch.cuda.memory_reserved'] = torch.cuda.memory_reserved(0)/1024/1024/1024 # GB
            logger['train_torch.cuda.max_memory_reserved'] = torch.cuda.max_memory_reserved(0)/1024/1024/1024 # GB
            print(f"Training loss @ step {step+1}: {logging_loss/logging_frequency}")
            eval_acc, eval_f1, eval_loss, eval_time = evaluate(model, testing_data_loader)
            logger['eval_accs'].append(eval_acc)
            logger['eval_f1s'].append(eval_f1)
            logger['eval_losses'].append(eval_loss)
            logger['eval_time'].append(eval_time+logger['eval_time'][-1])
            logger['eval_torch.cuda.memory_allocated'] = torch.cuda.memory_allocated(0)/1024/1024/1024 # GB
            logger['eval_torch.cuda.memory_reserved'] = torch.cuda.memory_reserved(0)/1024/1024/1024 # GB
            logger['eval_torch.cuda.max_memory_reserved'] = torch.cuda.max_memory_reserved(0)/1024/1024/1024 # GB
            logging_loss = 0
            mini_start_time = time.time()
    return epoch_loss / len(training_data_loader), time.time()-start_time


def evaluate(model: torch.nn.Module, test_data_loader: DataLoader):
    model.eval()
    model.to(device)
    eval_loss = 0
    correct_predictions = {i: 0 for i in range(2)}
    total_predictions = {i: 0 for i in range(2)}
    preds = []
    targets = []
    start_time = time.time()
    with torch.no_grad():
        for step, batch in enumerate(test_data_loader):
            batch = {key: value.to(device) for key, value in batch.items()}
            outputs = model(**batch)
            loss = outputs[0]
            eval_loss += loss.item()

            predictions = np.argmax(outputs[1].detach().cpu().numpy(), axis=1)
            preds.extend(predictions.tolist())
            targets.extend(batch["labels"].cpu().numpy().tolist())

            for target, prediction in zip(batch["labels"].cpu().numpy(), predictions):
                if target == prediction:
                    correct_predictions[target] += 1
                total_predictions[target] += 1
    accuracy = (100.0 * sum(correct_predictions.values())) / sum(total_predictions.values())
    f1 = f1_score(targets, preds)
    model.train()
    return accuracy, round(f1, 4), eval_loss / len(test_data_loader), time.time() - start_time


def save_logs(dictionary, log_dir, exp_id):
  log_dir = os.path.join(log_dir, exp_id)
  os.makedirs(log_dir, exist_ok=True)
  # Log arguments
  with open(os.path.join(log_dir, "args.json"), "w") as f:
    json.dump(dictionary, f, indent=2)

def save_model(model, log_dir, exp_id):
  log_dir = os.path.join(log_dir, exp_id)
  os.makedirs(log_dir, exist_ok=True)
  # Save model
  torch.save(model.state_dict(), f"assignment/models/model_{exp_id}.pt")

batch_size = 512

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, collate_fn=collate)

--> Device selected: cpu


### 6. Problem 3
Feel free to modify this code however it is convenient for you to produce a report except for the model parameters.

In [19]:
logging_frequency = 100
learning_rate = 1e-5
nb_epoch=5

for i in range(1, 2):
  experimental_setting = i

  if experimental_setting == 1:
    print("Setting 1: LSTM, no dropout, encoder only")
    model = ReviewClassifierLSTM(nb_classes=2, dropout=0, encoder_only=True)
  if experimental_setting == 2:
    print("Setting 2: LSTM, dropout, encoder only")
    model = ReviewClassifierLSTM(nb_classes=2, dropout=0.3, encoder_only=True)
  if experimental_setting == 3:
    print("Setting 3: LSTM, dropout, encoder-decoder, no attention")
    model = ReviewClassifierLSTM(nb_classes=2, dropout=0.3, encoder_only=False, with_attn=False)
  if experimental_setting == 4:
    print("Setting 4: LSTM, dropout, encoder-decoder, with attention")
    model = ReviewClassifierLSTM(nb_classes=2, dropout=0.3, encoder_only=False, with_attn=True)
  if experimental_setting == 5:
    print("Setting 5: Transformer, 2 layers, pre-normalization")
    model = ReviewClassifierTransformer(nb_classes=2, num_heads=4, num_layers=2, block='prenorm', dropout=0.3)
  if experimental_setting == 6:
    print("Setting 6: Transformer, 4 layers, pre-normalization")
    model = ReviewClassifierTransformer(nb_classes=2, num_heads=4, num_layers=4, block='prenorm', dropout=0.3)
  if experimental_setting == 7:
    print("Setting 7: Transformer, 2 layers, post-normalization")
    model = ReviewClassifierTransformer(nb_classes=2, num_heads=4, num_layers=2, block='postnorm', dropout=0.3)
  if experimental_setting == 8:
    nb_epoch = 2
    print("Setting 8: Fine-tuning BERT")
    model = ReviewClassifier(backbone="bert-base-uncased", backbone_hidden_size=768, nb_classes=2)
    for parameter in model.back_bone.parameters():
      parameter.requires_grad= False


  # setting up the optimizer
  optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate, eps=1e-8)
  model.to(device)
  logger = dict()
  logger['train_time'] = [0]
  logger['eval_time'] = [0]
  logger['train_losses'] = []
  logger['eval_accs'] = []
  logger['eval_f1s'] = []
  logger['eval_losses'] = []
  logger["epoch_train_loss"] = []
  logger["epoch_train_time"] = []
  logger["epoch_eval_loss"] = []
  logger["epoch_eval_time"] = []
  logger["epoch_eval_acc"] = []
  logger["epoch_eval_f1"] = []
  logger['train_torch.cuda.memory_allocated'] = []
  logger['train_torch.cuda.memory_reserved'] = []
  logger['train_torch.cuda.max_memory_reserved'] = []
  logger['eval_torch.cuda.memory_allocated'] = []
  logger['eval_torch.cuda.memory_reserved'] = []
  logger['eval_torch.cuda.max_memory_reserved'] = []
  logger['parameters'] = sum([p.numel() for p in model.back_bone.parameters() if p.requires_grad])

  for epoch in range(nb_epoch):
    print(f"Epoch {epoch+1}")
    if experimental_setting == 8 and epoch>1: #unfreezing layer 10 for fine-tuning
      for name, param in model.back_bone.named_parameters():
        if name.startswith("encoder.layer.11"):
            param.requires_grad = True
    train_loss, train_time = train_one_epoch(model, train_loader, optimizer, logging_frequency, test_loader, logger)
    print("Train, torch.cuda.memory_allocated: %fGB"%(torch.cuda.memory_allocated(0)/1024/1024/1024))
    print("Train, torch.cuda.memory_reserved: %fGB"%(torch.cuda.memory_reserved(0)/1024/1024/1024))
    print("Train, torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1024/1024/1024))
    eval_acc, eval_f1, eval_loss, eval_time  = evaluate(model, test_loader)
    print("Val, torch.cuda.memory_allocated: %fGB"%(torch.cuda.memory_allocated(0)/1024/1024/1024))
    print("Val, torch.cuda.memory_reserved: %fGB"%(torch.cuda.memory_reserved(0)/1024/1024/1024))
    print("Val, torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1024/1024/1024))
    logger["epoch_train_loss"].append(train_loss)
    logger["epoch_train_time"].append(train_time)
    logger["epoch_eval_loss"].append(eval_loss)
    logger["epoch_eval_time"].append(eval_time)
    logger["epoch_eval_acc"].append(eval_acc)
    logger["epoch_eval_f1"].append(eval_f1)
    print(f"    Epoch: {epoch+1} Loss/Test: {eval_loss}, Loss/Train: {train_loss}, Acc/Test: {eval_acc}, F1/Test: {eval_f1}, Train Time: {train_time}, Eval Time: {eval_time}")

  logger['train_time'] = logger['train_time'][1:]
  logger['eval_time'] = logger['eval_time'][1:]
  save_logs(logger, "assignment/log", str(experimental_setting))
  save_model(model, "assignment/models", str(experimental_setting))

Setting 1: LSTM, no dropout, encoder only
Epoch 1


KeyboardInterrupt: 

### 7. Augment the original reviews

In [27]:
from textattack.augmentation import Augmenter
from textattack.transformations import WordSwapQWERTY
from textattack.transformations import WordSwapExtend
from textattack.transformations import WordSwapContract
from textattack.transformations import WordSwapHomoglyphSwap
from textattack.transformations import CompositeTransformation
from textattack.transformations import WordSwapRandomCharacterDeletion
from textattack.transformations import WordSwapNeighboringCharacterSwap
from textattack.transformations import WordSwapRandomCharacterInsertion
from textattack.transformations import WordSwapRandomCharacterSubstitution

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/agathefernandesmachado/nltk_data'
    - '/Users/agathefernandesmachado/Downloads/deepL_assignment2/.venv/nltk_data'
    - '/Users/agathefernandesmachado/Downloads/deepL_assignment2/.venv/share/nltk_data'
    - '/Users/agathefernandesmachado/Downloads/deepL_assignment2/.venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [30]:
# Word-level Augmentations
# This technique involves replacing a word with a contracted form of it, such as replacing "cannot" with "can't".
word_swap_contract = True
#This likely involves extending a word by adding extra characters, such as replacing "good" with "goood".
word_swap_extend = False
# Homoglyphs are characters that look visually similar but have different Unicode representations. This technique involves replacing characters in words with visually similar characters, such as replacing "O" with "0".
word_swap_homoglyph_swap = False


# Character-level Augmentations
# This technique swaps neighboring characters within words. For example, "hello" might become "ehllo".
word_swap_neighboring_character_swap = True
# This likely involves replacing characters with adjacent characters on a QWERTY keyboard layout. For instance, "cat" might become "vat".
word_swap_qwerty = False
# This technique randomly deletes characters from words. For example, "house" might become "huse".
word_swap_random_character_deletion = False
# This involves randomly inserting characters into words. For instance, "book" might become "blook"
word_swap_random_character_insertion = False
# This technique randomly substitutes characters in words with other characters. For example, "apple" might become "apxle".
word_swap_random_character_substitution = False

# Check all the augmentations that you wish to apply!

# NOTE: Try applying each augmentation individually, and observe the changes.

# Apply augmentations
augmentations = []
if word_swap_contract:
  augmentations.append(WordSwapContract())
if word_swap_extend:
  augmentations.append(WordSwapExtend())
if word_swap_homoglyph_swap:
  augmentations.append(WordSwapHomoglyphSwap())
if word_swap_neighboring_character_swap:
  augmentations.append(WordSwapNeighboringCharacterSwap())
if word_swap_qwerty:
  augmentations.append(WordSwapQWERTY())
if word_swap_random_character_deletion:
  augmentations.append(WordSwapRandomCharacterDeletion())
if word_swap_random_character_insertion:
  augmentations.append(WordSwapRandomCharacterInsertion())
if word_swap_random_character_substitution:
  augmentations.append(WordSwapRandomCharacterSubstitution())

transformation = CompositeTransformation(augmentations)
augmenter = Augmenter(transformation=transformation,
                      transformations_per_example=1)


review = "I loved the food and the service was great!"
augmented_review = augmenter.augment(review)[0]
print("Augmented review:\n")
print(augmented_review)

NameError: name 'WordSwapContract' is not defined

In [34]:
import random

In [35]:
def getPrediction(text, model):
  """
  Outputs model prediction based on the input text.

  Args:
    text: String
      Input text

  Returns:
    item of pred: Iterable
      Prediction on the input text
  """
  inputs = tokenizer(text, padding="max_length", max_length=256,
                     truncation=True, return_tensors="pt")
  for key, value in inputs.items():
    inputs[key] = value.to(model.device)


  outputs = model(**inputs)
  logits = outputs.logits
  pred = torch.argmax(logits, dim=1)
  return pred.item()

In [36]:
models = [ReviewClassifierLSTM(nb_classes=2, dropout=0.3, encoder_only=True), 
          ReviewClassifierLSTM(nb_classes=2, dropout=0.3, encoder_only=False, with_attn=False),
          ReviewClassifierLSTM(nb_classes=2, dropout=0.3, encoder_only=False, with_attn=True),
          ReviewClassifierTransformer(nb_classes=2, num_heads=4, num_layers=4, block='prenorm', dropout=0.3),
          ReviewClassifierTransformer(nb_classes=2, num_heads=4, num_layers=2, block='postnorm', dropout=0.3)]

[{'word_swap_contract': True, 'word_swap_extend': True, 'word_swap_homoglyph_swap': False, 'word_swap_neighboring_character_swap': False, 'word_swap_qwerty': True, 'word_swap_random_character_deletion': True, 'word_swap_random_character_insertion': True, 'word_swap_random_character_substitution': True}, {'word_swap_contract': True, 'word_swap_extend': False, 'word_swap_homoglyph_swap': False, 'word_swap_neighboring_character_swap': True, 'word_swap_qwerty': True, 'word_swap_random_character_deletion': True, 'word_swap_random_character_insertion': False, 'word_swap_random_character_substitution': True}, {'word_swap_contract': True, 'word_swap_extend': True, 'word_swap_homoglyph_swap': True, 'word_swap_neighboring_character_swap': True, 'word_swap_qwerty': True, 'word_swap_random_character_deletion': True, 'word_swap_random_character_insertion': True, 'word_swap_random_character_substitution': True}, {'word_swap_contract': True, 'word_swap_extend': True, 'word_swap_homoglyph_swap': False

In [ ]:
n_samples_to_see = 1000
result_dict = {
    'configuration': [],
    'model': [],
    'f1_score': [],
    'accuracy': []
}

for j in range(10):
  word_swap_contract = True
  word_swap_extend = random.choice([True, False])
  word_swap_homoglyph_swap = True
  word_swap_neighboring_character_swap = random.choice([True, False])
  word_swap_qwerty = random.choice([True, False])
  word_swap_random_character_deletion = random.choice([True, False])
  word_swap_random_character_insertion = True
  word_swap_random_character_substitution = random.choice([True, False])
  config = {
            'word_swap_contract': word_swap_contract,
            'word_swap_extend': word_swap_extend,
            'word_swap_homoglyph_swap': word_swap_homoglyph_swap,
            'word_swap_neighboring_character_swap': word_swap_neighboring_character_swap,
            'word_swap_qwerty': word_swap_qwerty,
            'word_swap_random_character_deletion': word_swap_random_character_deletion,
            'word_swap_random_character_insertion': word_swap_random_character_insertion,
            'word_swap_random_character_substitution': word_swap_random_character_substitution
        }
  augmentations = []
  if word_swap_contract:
    augmentations.append(WordSwapContract())
  if word_swap_extend:
    augmentations.append(WordSwapExtend())
  if word_swap_homoglyph_swap:
    augmentations.append(WordSwapHomoglyphSwap())
  if word_swap_neighboring_character_swap:
    augmentations.append(WordSwapNeighboringCharacterSwap())
  if word_swap_qwerty:
    augmentations.append(WordSwapQWERTY())
  if word_swap_random_character_deletion:
    augmentations.append(WordSwapRandomCharacterDeletion())
  if word_swap_random_character_insertion:
    augmentations.append(WordSwapRandomCharacterInsertion())
  if word_swap_random_character_substitution:
    augmentations.append(WordSwapRandomCharacterSubstitution())

  transformation = CompositeTransformation(augmentations)
  augmenter = Augmenter(transformation=transformation,
                        transformations_per_example=1)

  for model in models:
    pred_labels = []
    true_labels = []
    for i in range(n_samples_to_see):
      text = dataset_test[i]["text"]
      label = dataset_test[i]["label"]
      augmented_text = augmenter.augment(text)[0]
      pred_label = getPrediction(augmented_text, model)
      pred_labels.append(pred_label)
      true_labels.append(label)
    accuracy = accuracy_score(true_labels, pred_labels)
    f1 = f1_score(true_labels, pred_labels)
    result_dict['configuration'].append(config)
    result_dict['model'].append(model)
    result_dict['f1_score'].append(f1_score)
    result_dict['accuracy'].append(accuracy)

save_logs(result_dict, "assignment/aug", 'aug')